In [3]:
import sys
import seaborn as sns
import os
import glob
from matplotlib import pyplot as plt
import numpy as np
import plot as pltfn
import network as netfn

%load_ext autoreload
%autoreload

# Define paths
#---------------------------------------------------------------------------
Fdrop = '/Users/dominicburrows/Dropbox/PhD/analysis/'
Fmac = '/Users/dominicburrows/Documents/'
F10t = '/Volumes/Dominic 10tb/'
Ffig = '/Users/dominicburrows/Dropbox/PhD/figures/'
Ftm = '/Volumes/Dominic Time Machine/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Create coordlist (only contain one fish)
#---------------------------------------------------------------------------
experiment = 'PTZ-WILDTYPE'
os.chdir(Fdrop + 'Project/' + experiment)
coordlist = sorted(glob.glob('*07*BLN*realcoord.npy'))
coordlist

['PTZ-WILDTYPE-07_2photon_sess-01-6dpf_BLN_run-01_realcoord.npy']

In [5]:
np.load(Fdrop + 'Project/' + experiment + os.sep + coordlist[0])

array([[152. , 171. ,   0. ],
       [188.5, 194. ,   0. ],
       [192. , 173. ,   0. ],
       ...,
       [309.5, 369. ,   9. ],
       [323. , 209. ,   9. ],
       [ 34. , 106. ,   9. ]])

In [9]:
from sklearn.cluster import KMeans

mcc = 15
cs = np.load(Fdrop + 'Project/' + experiment + os.sep + coordlist[0])
    
for pl in range(0,1):#np.unique(cs[:,2]):  # third dimension contains the plane index 
    print('Now working on plane ' + str(pl) + ' of ' + str(np.max(np.unique(cs[:,2]))))
    id       = np.where(cs[:,2] == pl)[0]
    n_clust  = int(np.max(id.shape) / mcc)
    #kmeans   = KMeans(n_clusters=n_clust, random_state=0).fit(cs[id,0:2])             

Now working on plane 0 of 9.0


In [10]:
n_clust

49

In [5]:
kmeans.labels_

array([ 4, 12, 18, 18, 38,  4, 36, 39, 18, 39, 20, 20, 40, 25, 27, 34, 25,
        9, 18,  4,  4, 38,  1, 40, 20, 44, 18, 20, 27, 40,  4, 21, 39,  9,
       25, 27, 40, 36, 39, 18, 12, 39, 30, 21, 20,  4, 27, 36, 36, 38, 18,
       43, 20, 20, 30,  4, 21, 44, 43, 40, 11, 12, 27, 34, 34, 34, 27, 39,
       36,  4,  1, 22, 33, 27, 11, 18,  4, 36,  6, 30, 11,  7, 27, 12, 20,
       11, 38,  0, 23, 39, 25, 12, 17, 36,  0, 25, 40,  1,  4, 38, 30, 36,
       10,  9, 23,  1, 20,  2, 43, 17, 43, 42,  6, 30, 37, 33, 20, 21, 27,
       21,  1, 39, 27, 29,  1, 20, 33, 39, 37,  6,  1, 34, 15, 11, 34, 24,
        2, 18, 42, 38, 14, 43, 25, 19, 40, 16, 32, 18, 39,  4, 44, 44, 38,
       39, 38, 18, 18,  9, 39, 21, 12, 20, 33, 39,  9,  0, 27, 20, 20, 30,
       27, 36, 38,  4,  0, 44,  4, 11, 40, 34, 18,  2, 40, 14, 21, 17, 22,
       30,  2, 36, 27, 25, 24, 18, 21,  9,  7,  0, 14, 21, 11,  6, 38, 42,
       27, 34,  4, 21,  1, 33,  9, 37, 16, 34, 10, 35, 13, 33,  5, 45, 27,
        6,  7, 29, 17, 13

# Functional node parcellation

## Spatial clusters

In [ ]:
# Run k-mean clustering on x-y coordinates to get spatially contiguous seed clusters
#-----------------------------------------------------------------------------------------
Fkmean = fn.kmeans(coordlist, Fdrop, experiment, mcc = 15)

In [ ]:
# Plot an illustrative example
#----------------------------------------------------------------------------------------
plane = 3

# Pull out the data from fish structure
#----------------------------------------------------------------------------------------
cs = Fish["Coordinates"]                  # 3D array of xyz coordinates
ci = Fish["Planes"][plane]["Index"]       # Index of plane coordinates in long list
ks = Fish["Planes"][plane]["Klabel"]      # Label from kmeans clustering

# Actual plotting routines
#----------------------------------------------------------------------------------------
plt.figure(figsize = (7.5,7.5))
plt.scatter(cs[ci,0], cs[ci,1], s = 20, c = ks, cmap = 'Set1')

# Further parcellate according to functional correlation structure

In [ ]:
# Calculate average traces within clusters
#----------------------------------------------------------------------------------------------
d      = Fish["Data"]
l      = Fish["Klabel"]
loc    = Fish["Coordinates"]
md,ml,mloc  = fn.fish_net_average(d,l,loc)

# Pack averaged data into Fish structure
#----------------------------------------------------------------------------------------------
Kdata = {"Label":ml, "Data":md, "Coordinates":mloc}
Fish.update({"Kdata":Kdata})

In [ ]:
# Plot an example traces
#----------------------------------------------------------------------------------------------
nplt  = 15
d     = Fish["Kdata"]["Data"][:nplt,:]
cm    = plt.get_cmap("Paired")

plt.figure(figsize=(15,15))
for i in range(nplt):
    plt.plot(d[i,:] + 1500*i, c=cm(1))

### _Find functionally related clusters_

In [ ]:
# Plot correlation matrix of drift corrected data
#---------------------------------------------------------------------------------
cor = np.corrcoef(Fish["Kdata"]["Data"])
plt.rc('figure', figsize=(7.5,7.5))
plt.matshow(cor, cmap='RdBu_r')

In [ ]:
# Run K-means clustering on correlation matrix 
#------------------------------------------------------------------------------------
kmeans   = KMeans(n_clusters=100, random_state=0).fit(cor)
kl       = kmeans.labels_
outplot  = 'corr'       # corr or plane

# Plot an example plane
#-------------------------------------------------------------------------------------
if outplot == 'plane':
    plane = 3
    cs    = Fish["Kdata"]["Coordinates"]
    pi    = np.where(cs[:,2] == plane)[0]

    plt.figure(figsize=(5,5))
    plt.scatter(cs[pi,0], cs[pi,1], c=kl[pi], cmap = 'magma')

# Plot correlation matrix ordered by k-means cluster
#------------------------------------------------------------------------------------
if outplot == 'corr':
    klstd   = np.argsort(kl)
    scor    = cor[klstd,:]
    scor    = scor[:,klstd] 
    plt.rc('figure', figsize=(7.5,7.5))
    plt.matshow(scor,cmap='RdBu_r')

### _Split up clusters that are too spatially separate_

In [ ]:
# Split up clusters according to spatial parameters
#-----------------------------------------------------------------------
cs  = Fish["Kdata"]["Coordinates"]
ps  = fn.fish_net_divconq(kl,cs)      # Parcel labelling

# Unpack to single neuron level
#-----------------------------------------------------------------------
cs = Fish["Coordinates"]
ks = Fish["Klabel"]
lb = np.zeros((cs.shape[0],1))
for i in range(cs.shape[0]): lb[i] = ps[int(ks[i])]
    
# Average within label groups
#----------------------------------------------------------------------
md,ml,mloc = fn.fish_net_average(Fish["Filt"], lb, cs)
Ps = ({"Label":ps, "Coordinates":mloc, "Data":md})
Fish.update({"Parcels":Ps})


In [ ]:
# Plot new categorisation
#-----------------------------------------------------------------------
fine_cs      = Fish["Coordinates"]
crse_cs      = Fish["Parcels"]["Coordinates"]
ks           = Fish["Klabel"]
fine_lb      = np.zeros((fine_cs.shape[0],1))
crse_lb      = np.zeros((crse_cs.shape[0],1))

for i in range(fine_cs.shape[0]): fine_lb[i] = ps[int(ks[i])]
for i in range(crse_cs.shape[0]): crse_lb[i] = Fish["Parcels"]["Label"][i]
    

cci    = np.where(Fish["Parcels"]["Coordinates"][:,2] == plane) 

# Actual plotting routines
#----------------------------------------------------------------------------------------
plane  = 5
fci    = Fish["Planes"][plane]["Index"]  
f, ax1 = plt.subplots(1, 1, sharey=True, figsize=(10,10))
ax1.scatter(fine_cs[fci,0], fine_cs[fci,1], s = 20, c = 'gray', cmap = 'Set1')
ax1.scatter(crse_cs[:,0], crse_cs[:,1], s = 250, c = crse_lb[:,0], cmap = 'Set1')


In [ ]:
# Plot example module traces
#---------------------------------------------------------------------------------------
plt.figure(figsize=(15,10))
plt.matshow(Fish["Parcels"]["Data"][:400,:1000], cmap='magma')
Fish["Parcels"]["Data"][:400,:1000].shape

In [ ]:
# Convert matrix into NetworkX Graph object to run module detection
#-------------------------------------------------------------------------------------
cor        = np.corrcoef(Fish["Parcels"]["Data"]); 
cor[cor < 0] = 0
G          = nx.from_numpy_array(cor)
partition  = community.best_partition(G)
M          = {"Label":np.asarray(partition.values())}
Fish.update({"Modules":M})